# Analiza kolumny system
Import danych

In [2]:
from collections import defaultdict
import pandas as pd


bestiary1 = pd.read_json("pathfinder_2e_data/pathfinder-bestiary.db", lines=True)
bestiary2 = pd.read_json("pathfinder_2e_data/pathfinder-bestiary-2.db", lines=True)
bestiary3 = pd.read_json("pathfinder_2e_data/pathfinder-bestiary-3.db", lines=True)
print(bestiary1.iloc[0])

_id                                                024PqcF8yMRBrPuq
img                        systems/pf2e/icons/default-icons/npc.svg
items             [{'_id': 'B7rzf0nBJmzg8x0y', 'img': 'systems/p...
name                                             Adult White Dragon
system            {'abilities': {'cha': {'mod': 1}, 'con': {'mod...
type                                                            npc
flags             {'core': {'sourceId': 'Compendium.pf2e.pathfin...
prototypeToken                                                  NaN
Name: 0, dtype: object


In [4]:
import numpy as np


b = [bestiary1, bestiary2, bestiary3]

bestiary = pd.concat(b, join="outer", axis=0).fillna(np.nan)

In [5]:
bestiary.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1206 entries, 0 to 364
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   _id             1206 non-null   object
 1   img             1206 non-null   object
 2   items           1206 non-null   object
 3   name            1206 non-null   object
 4   system          1206 non-null   object
 5   type            1206 non-null   object
 6   flags           1206 non-null   object
 7   prototypeToken  85 non-null     object
dtypes: object(8)
memory usage: 84.8+ KB


In [6]:
bestiary.head()

,_id,img,items,name,system,type,flags,prototypeToken
0,024PqcF8yMRBrPuq,systems/pf2e/icons/default-icons/npc.svg,"[{'_id': 'B7rzf0nBJmzg8x0y', 'img': 'systems/p...",Adult White Dragon,"{'abilities': {'cha': {'mod': 1}, 'con': {'mod...",npc,{'core': {'sourceId': 'Compendium.pf2e.pathfin...,NaN
1,05E3kkjoLZVjFOeO,systems/pf2e/icons/default-icons/npc.svg,"[{'_id': 'mYNDkS27YQRyy1F0', 'img': 'systems/p...",Brontosaurus,"{'abilities': {'cha': {'mod': 1}, 'con': {'mod...",npc,{'core': {'sourceId': 'Compendium.pf2e.pathfin...,NaN
2,05wwpHHsBlxBbdkN,systems/pf2e/icons/default-icons/npc.svg,"[{'_id': 'WkPeg600zGONsuJz', 'img': 'systems/p...",Giant Anaconda,"{'abilities': {'cha': {'mod': -2}, 'con': {'mo...",npc,{'core': {'sourceId': 'Compendium.pf2e.pathfin...,NaN
3,0FGz2eXm0SB04sJW,systems/pf2e/icons/default-icons/npc.svg,"[{'_id': 'joBYS96mXSnZC1WB', 'img': 'systems/p...",Ancient Green Dragon,"{'abilities': {'cha': {'mod': 6}, 'con': {'mod...",npc,{'core': {'sourceId': 'Compendium.pf2e.pathfin...,NaN
4,0HjVFx8qIKDCfblg,systems/pf2e/icons/default-icons/npc.svg,"[{'_id': 'xlUnQcjrxeMXu5Cd', 'img': 'systems/p...",Lantern Archon,"{'abilities': {'cha': {'mod': 1}, 'con': {'mod...",npc,{'core': {'sourceId': 'Compendium.pf2e.pathfin...,NaN


```unpack_column``` - Funkcja która rozpakowuje słownik i zmienia go w dataframe. Printuje pierwszą wartość tabeli i długość

In [7]:
def unpack_column(df, column_name):
    new_df = df[column_name].apply(pd.Series)
    print(new_df.iloc[0])
    print("Len =", len(new_df.iloc[0]))
    return new_df

In [8]:
bestiary_system = bestiary["system"].apply(pd.Series)
print(bestiary_system.iloc[0])
print("Len = ", len(bestiary_system.iloc[0]))

abilities     {'cha': {'mod': 1}, 'con': {'mod': 5}, 'dex': ...
attributes    {'ac': {'details': '', 'value': 29}, 'allSaves...
details       {'alignment': {'value': 'CE'}, 'blurb': '', 'c...
resources                     {'focus': {'max': 1, 'value': 1}}
saves         {'fortitude': {'saveDetail': '', 'value': 22},...
traits        {'languages': {'custom': '', 'selected': [], '...
schema        {'version': 0.8270000000000001, 'lastMigration...
Name: 0, dtype: object
Len =  7


## Abilities

In [11]:
abilities = unpack_column(bestiary_system, "abilities")

cha    {'mod': 1}
con    {'mod': 5}
dex    {'mod': 2}
int    {'mod': 1}
str    {'mod': 7}
wis    {'mod': 2}
Name: 0, dtype: object
Len = 6


Sprawdzenie jakie klucze występują w słownikach - czy tylko ```mod```?

In [12]:
def check_diff_values(df, values):
    return (
        len(
            [
                _
                for i, row in df.iterrows()
                for col in df.columns
                for val in values
                if len(row[col]) != len(values) or val not in row[col].keys()
            ]
        )
        > 0
    )

In [13]:
print(
    len(
        [
            _
            for i, row in abilities.iterrows()
            for col in abilities.columns
            if len(row[col]) != 1 or "mod" not in row[col].keys()
        ]
    )
)
print("Czy ją tu inne wartości oprócz 'mod'?", check_diff_values(abilities, ["mod"]))

0
Czy ją tu inne wartości oprócz 'mod'? False


Same mody, więc możnachyba zmienić te słowniki na wartość

In [14]:
abilities.head()

,cha,con,dex,int,str,wis
0,{'mod': 1},{'mod': 5},{'mod': 2},{'mod': 1},{'mod': 7},{'mod': 2}
1,{'mod': 1},{'mod': 5},{'mod': 0},{'mod': -4},{'mod': 9},{'mod': 2}
2,{'mod': -2},{'mod': 6},{'mod': 3},{'mod': -4},{'mod': 7},{'mod': 3}
3,{'mod': 6},{'mod': 5},{'mod': 4},{'mod': 6},{'mod': 7},{'mod': 5}
4,{'mod': 1},{'mod': 1},{'mod': 3},{'mod': -1},{'mod': -5},{'mod': 1}


### Zamiana słowników na wartości *mod*

In [18]:
from copy import deepcopy


abilities_backup = deepcopy(abilities)

Na razie nie znalazłam ładniejszego sposobu na podmianę

In [19]:
for col in abilities.columns:
    for i, row in abilities.iterrows():
        row[col] = row[col].get("mod")

In [20]:
abilities.head()

,cha,con,dex,int,str,wis
0,1,5,2,1,7,2
1,1,5,0,-4,9,2
2,-2,6,3,-4,7,3
3,6,5,4,6,7,5
4,1,1,3,-1,-5,1


**abilities do uwzględnienia w predykcji**

## Analiza attributes

In [21]:
attributes = unpack_column(bestiary_system, "attributes")

ac                                  {'details': '', 'value': 29}
allSaves           {'value': '+1 status to all saves vs. magic'}
hp             {'details': '', 'max': 215, 'temp': 0, 'value'...
immunities     [{'type': 'cold'}, {'type': 'paralyzed'}, {'ty...
initiative                             {'ability': 'perception'}
perception                                         {'value': 20}
speed          {'details': 'climb on ice only', 'otherSpeeds'...
weaknesses                       [{'type': 'fire', 'value': 10}]
resistances                                                  NaN
hardness                                                     NaN
adjustment                                                   NaN
Name: 0, dtype: object
Len = 11


In [22]:
attributes.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1206 entries, 0 to 364
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ac           1206 non-null   object 
 1   allSaves     1206 non-null   object 
 2   hp           1206 non-null   object 
 3   immunities   637 non-null    object 
 4   initiative   1206 non-null   object 
 5   perception   1206 non-null   object 
 6   speed        1206 non-null   object 
 7   weaknesses   463 non-null    object 
 8   resistances  344 non-null    object 
 9   hardness     10 non-null     object 
 10  adjustment   0 non-null      float64
dtypes: float64(1), object(10)
memory usage: 113.1+ KB


Brakujące wartości w kolumnach:
* immunities
* weaknesses
* resistance
W *adjustments* nic nie ma

In [23]:
attributes.head()

,ac,allSaves,hp,immunities,initiative,perception,speed,weaknesses,resistances,hardness,adjustment
0,"{'details': '', 'value': 29}",{'value': '+1 status to all saves vs. magic'},"{'details': '', 'max': 215, 'temp': 0, 'value'...","[{'type': 'cold'}, {'type': 'paralyzed'}, {'ty...",{'ability': 'perception'},{'value': 20},"{'details': 'climb on ice only', 'otherSpeeds'...","[{'type': 'fire', 'value': 10}]",NaN,NaN,NaN
1,"{'details': '', 'value': 28}",{'value': ''},"{'details': '', 'max': 220, 'temp': 0, 'value'...",NaN,{'ability': 'perception'},{'value': 16},"{'otherSpeeds': [], 'value': 35}",NaN,NaN,NaN,NaN
2,"{'details': '', 'value': 25}",{'value': ''},"{'details': '', 'max': 175, 'temp': 0, 'value'...",NaN,{'ability': 'perception'},{'value': 17},"{'otherSpeeds': [{'type': 'climb', 'value': 30...",NaN,NaN,NaN,NaN
3,"{'details': '', 'value': 41}",{'value': '+1 status to all saves vs. magic'},"{'details': '', 'max': 315, 'temp': 0, 'value'...","[{'type': 'paralyzed'}, {'type': 'poison'}, {'...",{'ability': 'perception'},{'value': 30},"{'details': 'trackless step, woodland stride',...",NaN,NaN,NaN,NaN
4,"{'details': '', 'value': 16}",{'value': ''},"{'details': '', 'max': 20, 'temp': 0, 'value':...",NaN,{'ability': 'perception'},{'value': 6},"{'otherSpeeds': [{'type': 'fly', 'value': 40}]...","[{'type': 'evil', 'value': 3}]","[{'type': 'fire', 'value': 3}]",NaN,NaN


Masa jakiś nowych informacji: znów słowniki

### Analiza attributes: ac

In [24]:
print(attributes.ac)

0      {'details': '', 'value': 29}
1      {'details': '', 'value': 28}
2      {'details': '', 'value': 25}
3      {'details': '', 'value': 41}
4      {'details': '', 'value': 16}
                   ...             
360    {'details': '', 'value': 46}
361    {'details': '', 'value': 15}
362    {'details': '', 'value': 33}
363    {'details': '', 'value': 22}
364    {'details': '', 'value': 24}
Name: ac, Length: 1206, dtype: object


In [25]:
print("Czy są inne kolumny niż 'details' oraz 'vaue'?")
check_diff_values(pd.DataFrame(data=attributes.ac), ["details", "value"])

Czy są inne kolumny niż 'details' oraz 'vaue'?


False

In [26]:
def find_other_values(df, values):
    result = []
    for i, row in df.iterrows():
        for col in df:
            if len(row[col]) != len(values):
                # print(row[col])
                result.append(row[col])
                continue
            for val in values:
                if val not in row[col].keys() > 0:
                    result.append(row[col])
    return result

In [27]:
attributes_ac_details = {i: row["details"] for i, row in attributes.ac.iteritems()}

In [45]:
def return_zero():
    return 0


def print_dictionery(dict):
    df = pd.DataFrame(
        data={
            "value": [key for key in dict.keys()],
            "number": [dict.get(key) for key in dict.keys()],
        }
    )
    display(df)


def check_not_null_val(df, col, val=""):
    not_null_dict = defaultdict(return_zero)
    for i, row in df.iterrows():
        v = row.get(col)
        if v != "" and v is not None and v != val:
            not_null_dict[v] += 1
    print("Diff Not null: ", len(not_null_dict))
    print("All null values: ", sum(item for i, item in not_null_dict.items()))
    print_dictionery(not_null_dict)
    # return not_null_dict

In [46]:
check_not_null_val(pd.DataFrame(data=unpack_column(attributes, "ac")), "details")

details      
value      29
Name: 0, dtype: object
Len = 2
Diff Not null:  24
All null values:  59


,value,number
0,all-around vision,25
1,(14 when broken); construct armor,1
2,(18 with shield raised),1
3,22 with shield raised,2
4,(15 when broken); construct armor,1
5,(20 with shield raised),2
6,(21 with shield raised),2
7,(22 against prey; see Hunt Prey),1
8,+1 status vs. evil creatures,2
9,(22 with shield raised),1


Można olać całość i uwzglednic jedynie **value**

Przynajmniej tak mi się wydaje

**ac value do uwzględnienia w predykcji**

### Analiza attributes: allSaves

In [47]:
print(attributes.allSaves)

0          {'value': '+1 status to all saves vs. magic'}
1                                          {'value': ''}
2                                          {'value': ''}
3          {'value': '+1 status to all saves vs. magic'}
4                                          {'value': ''}
                             ...                        
360        {'value': '+1 status to all saves vs. magic'}
361                                        {'value': ''}
362                                        {'value': ''}
363    {'value': '+1 status to all saves vs. divine a...
364                                        {'value': ''}
Name: allSaves, Length: 1206, dtype: object


In [48]:
print("czy w attributes allSaves występują inne dane niż 'value'?")
check_diff_values(pd.DataFrame(data=attributes.allSaves), ["value"])

czy w attributes allSaves występują inne dane niż 'value'?


False

In [49]:
check_not_null_val(pd.DataFrame(data=unpack_column(attributes, "allSaves")), "value")

value    +1 status to all saves vs. magic
Name: 0, dtype: object
Len = 1
Diff Not null:  50
All null values:  302


,value,number
0,+1 status to all saves vs. magic,207
1,guarded thoughts,1
2,+4 status to all saves vs. fear and dreams,1
3,+2 status to all saves vs. magic,8
4,+1 status to all saves vs. positive,8
5,"+1 status to all saves vs. magic, +2 status to...",3
6,"+1 status to all saves vs. magic, +4 status to...",7
7,+1 status to all saves vs. death effects,1
8,+4 status to all saves vs. fear,1
9,"+2 status to all saves vs. magic, -2 to all sa...",1


Nie wiem jak to uwzględnić

Gdyby to było w jakiejś postaci typu tabela kategorii i wartość dodana np. cold: +2

ale nie jest

więc chyba można olać

### Analiza attributes: *hp*

In [50]:
attributes_hp = unpack_column(attributes, "hp")

details       
max        215
temp         0
value      215
Name: 0, dtype: object
Len = 4


In [51]:
attributes_hp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1206 entries, 0 to 364
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   details  1206 non-null   object
 1   max      1206 non-null   int64 
 2   temp     1206 non-null   int64 
 3   value    1206 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 47.1+ KB


In [52]:
attributes_hp.head()

,details,max,temp,value
0,,215,0,215
1,,220,0,220
2,,175,0,175
3,,315,0,315
4,,20,0,20


In [53]:
hp_no_null_det = check_not_null_val(attributes_hp, "details")

Diff Not null:  84
All null values:  195


,value,number
0,fast healing 5,2
1,negative healing,67
2,"negative healing, rejuvenation",6
3,regeneration 15 (deactivated by cold iron),3
4,regeneration 10 (deactivated by cold iron),4
...,...,...
79,primal purpose,1
80,regeneration 50 (deactivated by fire),1
81,regeneration 5 (deactivated by acid or fire),1
82,Regeneration 10 (deactivated by piercing),1


analogicznie jak powyżej

In [54]:
check_not_null_val(attributes_hp, "temp", 0)

Diff Not null:  0
All null values:  0


,value,number


In [69]:
attributes_hp.loc[attributes_hp["temp"] != 0].size

0

temp to zawsze "wartość pusta" => 0 -można olać

**Wydaje mi się że stąd można "ukraść" max oraz value**

*Podsumowanie*

* details - nie będziemy uwzględniać
* max - można, czemu nie (int + brak wartości pustych)
* temp - i tak same 0 w bestiariuszach, więc jak na ten moment to raczej nie
* **value - na pewno bierzemy**

### Analiza attributes: *immunities*

In [71]:
immunities = unpack_column(attributes, "immunities")

C:\Users\jolka\AppData\Local\Temp\ipykernel_14364\685769781.py:2: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_df = df[column_name].apply(pd.Series)


0          {'type': 'cold'}
1     {'type': 'paralyzed'}
2         {'type': 'sleep'}
3                       NaN
4                       NaN
5                       NaN
6                       NaN
7                       NaN
8                       NaN
9                       NaN
10                      NaN
11                      NaN
12                      NaN
13                      NaN
14                      NaN
15                      NaN
Name: 0, dtype: object
Len = 16


In [107]:
immunities.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,{'type': 'cold'},{'type': 'paralyzed'},{'type': 'sleep'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,{'type': 'paralyzed'},{'type': 'poison'},{'type': 'sleep'},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
print("NaN:", end=" ")
immunities.info()

NaN: <class 'pandas.core.frame.DataFrame'>
Int64Index: 1206 entries, 0 to 364
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       632 non-null    object
 1   1       492 non-null    object
 2   2       395 non-null    object
 3   3       253 non-null    object
 4   4       193 non-null    object
 5   5       106 non-null    object
 6   6       69 non-null     object
 7   7       64 non-null     object
 8   8       61 non-null     object
 9   9       61 non-null     object
 10  10      57 non-null     object
 11  11      57 non-null     object
 12  12      36 non-null     object
 13  13      23 non-null     object
 14  14      4 non-null      object
 15  15      2 non-null      object
dtypes: object(16)
memory usage: 160.2+ KB


Niemal połowa potworków (1206 - 632) nie ma nic wpisane w tej kolumnie

Wydaje mi się, że się raczej nie przyda

Aczkolwiek można porozbijać na kolumny - może coś da

### Analiza attributes: *initiative*

In [90]:
initiative = unpack_column(attributes, "initiative")

ability    perception
Name: 0, dtype: object
Len = 1


In [91]:
initiative.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1206 entries, 0 to 364
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   ability  1206 non-null   object
dtypes: object(1)
memory usage: 18.8+ KB


In [92]:
initiative.head()

,ability
0,perception
1,perception
2,perception
3,perception
4,perception


In [94]:
initiative.loc[initiative["ability"] != "perception"]

,ability
85,dec


In [102]:
bestiary.iloc[85, :]

_id                                                BRo0snV2sH6TFuh6
img                        systems/pf2e/icons/default-icons/npc.svg
items             [{'_id': '7aI3LWA9cnoKonHZ', 'img': 'systems/p...
name                                                       Glabrezu
system            {'abilities': {'cha': {'mod': 7}, 'con': {'mod...
type                                                            npc
flags             {'core': {'sourceId': 'Compendium.pf2e.pathfin...
prototypeToken                                                  NaN
Name: 85, dtype: object

In [103]:
bestiary.iloc[85, 4].get("details").get("level")

{'value': 13}

Wszystkie potworki oprócz Glabrezu mają wpisane "perception"

Też raczej można olać

### Analiza attributes: *perception*

In [104]:
perception = unpack_column(attributes, "perception")

value    20
Name: 0, dtype: int64
Len = 1


In [105]:
perception.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1206 entries, 0 to 364
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   value   1206 non-null   int64
dtypes: int64(1)
memory usage: 51.1 KB


In [106]:
perception.head()

,value
0,20
1,16
2,17
3,30
4,6


Dane w "przyjaznej" formie, są uwzględniane w statystykach - *mogą się przydać*

**można uwzględnić perception w predykcji**

### Analiza attributes: *speed*

In [108]:
speed = unpack_column(attributes, "speed")

details                                        climb on ice only
otherSpeeds    [{'type': 'climb', 'value': 30}, {'type': 'fly...
value                                                       30.0
Name: 0, dtype: object
Len = 3


In [109]:
speed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1206 entries, 0 to 364
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   details      124 non-null    object 
 1   otherSpeeds  1206 non-null   object 
 2   value        1205 non-null   float64
dtypes: float64(1), object(2)
memory usage: 70.0+ KB


In [110]:
speed.head()

,details,otherSpeeds,value
0,climb on ice only,"[{'type': 'climb', 'value': 30}, {'type': 'fly...",30.0
1,NaN,[],35.0
2,NaN,"[{'type': 'climb', 'value': 30}, {'type': 'swi...",30.0
3,"trackless step, woodland stride","[{'type': 'fly', 'value': 200}, {'type': 'swim...",50.0
4,NaN,"[{'type': 'fly', 'value': 40}]",0.0


Wydaje mi się, żę też to można uwzględnić tylko, że trzeba to jakoś rozbić na kolumny i zdecydować co zrobić z wartościami brakującymi - pierwszy strzał to dać tam 0 (jak ktoś nie umie latać to fly speed == 0)

**do uwzględnienia**

### Analiza attributes: *weaknesses*

In [111]:
weaknesses = unpack_column(attributes, "weaknesses")

0    {'type': 'fire', 'value': 10}
1                              NaN
2                              NaN
3                              NaN
Name: 0, dtype: object
Len = 4


In [112]:
weaknesses.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1206 entries, 0 to 364
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       463 non-null    object
 1   1       127 non-null    object
 2   2       15 non-null     object
 3   3       2 non-null      object
dtypes: object(4)
memory usage: 79.4+ KB


In [113]:
weaknesses.head()

,0,1,2,3
0,"{'type': 'fire', 'value': 10}",NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,"{'type': 'evil', 'value': 3}",NaN,NaN,NaN


In [118]:
weaknesses[pd.notnull(weaknesses[3])]

,0,1,2,3
287,"{'type': 'cold', 'value': 15}","{'type': 'cold-iron', 'value': 15}","{'type': 'good', 'value': 15}","{'type': 'slashing', 'value': 10}"
233,"{'type': 'area-damage', 'value': 5}","{'type': 'positive', 'value': 5}","{'type': 'slashing', 'value': 5}","{'type': 'splash-damage', 'value': 2}"


In [121]:
weaknesses[pd.isnull(weaknesses[0])].shape[0]

743

Większość potworków nie ma słabości, około 10% ma dwie, a więcej (do 4) zdarza się sporadycznie

*Zawsze można spróbować to uwzględnić może pomoże*

### Analiza attributes: *resistances*

In [122]:
resistances = unpack_column(attributes, "resistances")

0    NaN
1    NaN
2    NaN
3    NaN
4    NaN
5    NaN
Name: 0, dtype: object
Len = 6


In [123]:
resistances.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1206 entries, 0 to 364
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       344 non-null    object
 1   1       137 non-null    object
 2   2       48 non-null     object
 3   3       22 non-null     object
 4   4       15 non-null     object
 5   5       6 non-null      object
dtypes: object(6)
memory usage: 98.2+ KB


In [124]:
resistances.head()

,0,1,2,3,4,5
0,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'type': 'fire', 'value': 3}",NaN,NaN,NaN,NaN,NaN


In [126]:
resistances[pd.notnull(resistances[5])]

,0,1,2,3,4,5
21,"{'type': 'bludgeoning', 'value': 5}","{'type': 'cold', 'value': 10}","{'type': 'electricity', 'value': 10}","{'type': 'fire', 'value': 10}","{'type': 'piercing', 'value': 10}","{'type': 'slashing', 'value': 10}"
104,"{'type': 'bludgeoning', 'value': 6}","{'type': 'cold', 'value': 12}","{'type': 'electricity', 'value': 12}","{'type': 'fire', 'value': 12}","{'type': 'piercing', 'value': 12}","{'type': 'slashing', 'value': 12}"
206,"{'type': 'bludgeoning', 'value': 6}","{'type': 'cold', 'value': 12}","{'type': 'electricity', 'value': 12}","{'type': 'fire', 'value': 12}","{'type': 'piercing', 'value': 12}","{'type': 'slashing', 'value': 12}"
220,"{'type': 'bludgeoning', 'value': 5}","{'type': 'cold', 'value': 10}","{'type': 'electricity', 'value': 10}","{'type': 'fire', 'value': 10}","{'type': 'piercing', 'value': 10}","{'type': 'slashing', 'value': 10}"
276,"{'type': 'bludgeoning', 'value': 5}","{'type': 'cold', 'value': 10}","{'type': 'electricity', 'value': 10}","{'type': 'fire', 'value': 10}","{'type': 'piercing', 'value': 10}","{'type': 'slashing', 'value': 10}"
279,"{'type': 'bludgeoning', 'value': 5}","{'type': 'cold', 'value': 10}","{'type': 'electricity', 'value': 10}","{'type': 'fire', 'value': 10}","{'type': 'piercing', 'value': 10}","{'type': 'slashing', 'value': 10}"


*To też można uwzględnić*

Potencjalnie dużo kolumn może się stwożyć - podobnie jak wyżej tam gdzie brak wartości proponuję 0

### Analiza attributes: *hardness*

In [127]:
hardness = unpack_column(attributes, "hardness")

0       NaN
value   NaN
Name: 0, dtype: float64
Len = 2


In [128]:
hardness.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1206 entries, 0 to 364
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       0 non-null      float64
 1   value   10 non-null     float64
dtypes: float64(2)
memory usage: 60.6 KB


In [130]:
hardness.head()

,0,value
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


In [140]:
hardness[pd.notnull(hardness[0])]

,0,value


Nie wiem co się stało przy rozpakowywaniu, ale tu są tylko *values*

**Wydaje mi się, że to też można użyć**

### Analiza attributes: *adjustment*

In [141]:
adjustment = unpack_column(attributes, "adjustment")

0   NaN
Name: 0, dtype: float64
Len = 1


In [142]:
adjustment.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1206 entries, 0 to 364
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       0 non-null      float64
dtypes: float64(1)
memory usage: 51.1 KB


In [143]:
adjustment.head()

,0
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN


Tutaj są same nulle, więc można usunąć

## Podsumowując:
* Abilities - wszystkie wartości do uwzględnienia
* Attributes:
    * ac - value do uwzględnienia
    * allSaves - raczej nie
    * hp - value do uwzględnienia + ewentualnie max
    * immunities - jak na razie nie, ale można spróbować
    * initiative - raczej nie jest potrzebne
    * perception - można uwzględnić
    * speed - też można spróbować uwzględnić
    * weaknesses - do tego nie jestem w 100% przekonana, ale zawsze można dodać
    * resistances - analogicznie jak wyżej
    * hardness - myślę, że można uwzględnić
    * adjustment - tutaj nic nie ma (dosłownie)

In [70]:
attributes.head()

,ac,allSaves,hp,immunities,initiative,perception,speed,weaknesses,resistances,hardness,adjustment
0,"{'details': '', 'value': 29}",{'value': '+1 status to all saves vs. magic'},"{'details': '', 'max': 215, 'temp': 0, 'value'...","[{'type': 'cold'}, {'type': 'paralyzed'}, {'ty...",{'ability': 'perception'},{'value': 20},"{'details': 'climb on ice only', 'otherSpeeds'...","[{'type': 'fire', 'value': 10}]",NaN,NaN,NaN
1,"{'details': '', 'value': 28}",{'value': ''},"{'details': '', 'max': 220, 'temp': 0, 'value'...",NaN,{'ability': 'perception'},{'value': 16},"{'otherSpeeds': [], 'value': 35}",NaN,NaN,NaN,NaN
2,"{'details': '', 'value': 25}",{'value': ''},"{'details': '', 'max': 175, 'temp': 0, 'value'...",NaN,{'ability': 'perception'},{'value': 17},"{'otherSpeeds': [{'type': 'climb', 'value': 30...",NaN,NaN,NaN,NaN
3,"{'details': '', 'value': 41}",{'value': '+1 status to all saves vs. magic'},"{'details': '', 'max': 315, 'temp': 0, 'value'...","[{'type': 'paralyzed'}, {'type': 'poison'}, {'...",{'ability': 'perception'},{'value': 30},"{'details': 'trackless step, woodland stride',...",NaN,NaN,NaN,NaN
4,"{'details': '', 'value': 16}",{'value': ''},"{'details': '', 'max': 20, 'temp': 0, 'value':...",NaN,{'ability': 'perception'},{'value': 6},"{'otherSpeeds': [{'type': 'fly', 'value': 40}]...","[{'type': 'evil', 'value': 3}]","[{'type': 'fire', 'value': 3}]",NaN,NaN
